## NewsAnalysisAgent
This agent reads news headlines about a stock, asks GPT-4 "are these headlines good or bad news?", gets back a sentiment score, and packages it in a standardized format for the orchestrator to use.
For example: if someone whats get sentiment from new, this agent is  reading news and telling if it's bullish or bearish.

## EarningsAnalysisAgent
The Earnings Analysis Agent examines a company's financial performance (revenue, profits, earnings per share) and determines if the fundamentals are strong by comparing actual results against analyst expectations. It sends this financial data to GPT-4, which returns a score (-1 to +1) indicating whether the company's financials suggest it's a good or weak investment.

## MarketSignalsAgent
The MarketSignalsAgent performs technical analysis by examining stock price patterns, trading volume, and technical indicators (like moving averages, RSI, MACD) to identify trends and momentum. It sends this technical data to GPT-4, which returns a score (-1 to +1) indicating whether the stock's price action suggests a bullish or bearish trend based on chart patterns and trading signals.

## RiskAssessmentAgent
The RiskAssessmentAgent evaluates investment risk by analyzing metrics like beta (volatility), Value at Risk, Sharpe ratio, and sector correlation to determine how risky a stock is for a portfolio. It sends these risk metrics to GPT-4, which returns a risk score (0 to 1, where 0 is low risk and 1 is high risk) along with warnings about potential portfolio concentration or volatility issues.

## SynthesisAgent
The SynthesisAgent acts as the "decision maker" that takes all the individual agent analyses (news sentiment, earnings strength, technical signals, risk level) and combines them into a single investment recommendation (STRONG BUY, BUY, HOLD, SELL, STRONG SELL). It sends a summary of all agent scores and findings to GPT-4, which weighs the different perspectives and returns a final actionable recommendation with confidence level and supporting reasoning.

## CritiqueAgent

The CritiqueAgent acts as a "quality control checker" that reviews the final investment recommendation to catch mistakes, biases, or missing information before presenting it to the user. It examines the SynthesisAgent's recommendation, asks GPT-4 to identify logical flaws or gaps in reasoning, and can adjust the confidence level downward if it finds issues (like  "didn't consider macroeconomic factors"), ensuring the final output is reliable and well-reasoned.


In [88]:
"""
AGENT PROCESSING SYSTEM
Multi-Agent Financial Analysis
"""

from openai import OpenAI
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
import json
from datetime import datetime
import os
import time
import logging

# ============================================================================
# SETUP API KEYS (Open AI keys before running)
# ============================================================================

import os

# Alpha Vantage
os.environ["ALPHAVANTAGE_API_KEY"] = "BVGUKZR1MHVS0T6B"

# OpenAI - REPLACE WITH YOUR KEY
os.environ["OPENAI_API_KEY"] = "sk-proj-"  # REPLACE BEFORE RUNNING

# Initialize OpenAI client
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

print("API keys configured")

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


API keys configured


In [89]:
# ============================================================================
# DATA STRUCTURES
# ============================================================================

@dataclass
class AgentResponse:
    """Standard response format from all agents"""
    agent_name: str
    analysis: str
    score: float
    confidence: float
    key_factors: List[str]
    timestamp: str
    execution_time: float = 0.0  # NEW: Track execution time
    token_usage: Optional[Dict[str, int]] = None  # NEW: Track token usage


@dataclass
class AgentMetrics:
    """Track agent performance metrics"""
    agent_name: str
    total_calls: int = 0
    successful_calls: int = 0
    failed_calls: int = 0
    total_execution_time: float = 0.0
    average_execution_time: float = 0.0
    total_tokens_used: int = 0


In [90]:

# ============================================================================
# SHARED CONTEXT MANAGER
# ============================================================================

class SharedContext:
    """Allows agents to share insights with each other"""

    def __init__(self):
        self.context = {}
        self.history = []

    def add_insight(self, agent_name: str, key: str, value: Any):
        """Agent shares an insight"""
        self.context[key] = {
            'value': value,
            'from_agent': agent_name,
            'timestamp': datetime.now().isoformat()
        }
        self.history.append({
            'agent': agent_name,
            'key': key,
            'value': value,
            'timestamp': datetime.now().isoformat()
        })
        logger.info(f"{agent_name} shared insight: {key}")

    def get_insight(self, key: str) -> Optional[Any]:
        """Retrieve an insight"""
        return self.context.get(key, {}).get('value')

    def get_all_insights(self) -> Dict:
        """Get all shared insights"""
        return self.context

    def clear(self):
        """Clear all context"""
        self.context = {}
        self.history = []


# Initialize shared context for all agents
shared_context = SharedContext()



In [91]:
# ============================================================================
# BASE AGENT
# ============================================================================

class BaseAgent:
    """Enhanced base class for all financial agents with memory, retry, and metrics"""

    def __init__(self, agent_name: str, model: str = "gpt-4o-mini"):
        self.agent_name = agent_name
        self.model = model
        self.memory = []  # Conversation history
        self.metrics = AgentMetrics(agent_name=agent_name)  # NEW: Performance metrics
        self.shared_context = shared_context  # NEW: Shared context with other agents
        logger.info(f"Initialized {agent_name}")

    def call_llm(self, system_prompt: str, user_message: str, max_retries: int = 3) -> tuple:
        """
        Call LLM with error handling, retry logic, and token tracking
        Returns: (response_text, token_usage)
        """
        for attempt in range(max_retries):
            try:
                start_time = time.time()

                response = client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_message}
                    ],
                    temperature=0.3,
                    max_tokens=800
                )

                elapsed = time.time() - start_time

                # Extract token usage
                token_usage = {
                    'prompt_tokens': response.usage.prompt_tokens,
                    'completion_tokens': response.usage.completion_tokens,
                    'total_tokens': response.usage.total_tokens
                }

                # Update metrics
                self.metrics.total_tokens_used += token_usage['total_tokens']

                logger.info(f"{self.agent_name} LLM call completed in {elapsed:.2f}s, tokens: {token_usage['total_tokens']}")

                return response.choices[0].message.content, token_usage

            except Exception as e:
                if attempt == max_retries - 1:
                    logger.error(f"{self.agent_name} failed after {max_retries} attempts: {str(e)}")
                    return f"Error processing request: {str(e)}", None

                wait_time = 2 ** attempt  # Exponential backoff
                logger.warning(f"{self.agent_name} retry {attempt + 1}/{max_retries}, waiting {wait_time}s")
                time.sleep(wait_time)

    def add_to_memory(self, interaction: Dict):
        """Store conversation history"""
        self.memory.append({
            'timestamp': datetime.now().isoformat(),
            'input': interaction.get('input'),
            'output': interaction.get('output'),
            'metadata': interaction.get('metadata', {})
        })
        logger.debug(f"{self.agent_name} added to memory (total: {len(self.memory)})")

    def get_context(self, last_n: int = 5) -> List[Dict]:
        """Retrieve recent context from memory"""
        return self.memory[-last_n:] if self.memory else []

    def share_insight(self, key: str, value: Any):
        """Share an insight with other agents via shared context"""
        self.shared_context.add_insight(self.agent_name, key, value)

    def get_shared_insights(self) -> Dict:
        """Get insights shared by other agents"""
        return self.shared_context.get_all_insights()

    def update_metrics(self, success: bool, execution_time: float):
        """Update agent performance metrics"""
        self.metrics.total_calls += 1
        if success:
            self.metrics.successful_calls += 1
        else:
            self.metrics.failed_calls += 1

        self.metrics.total_execution_time += execution_time
        self.metrics.average_execution_time = (
            self.metrics.total_execution_time / self.metrics.total_calls
        )

    def get_metrics(self) -> AgentMetrics:
        """Get agent performance metrics"""
        return self.metrics

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Override in each specialized agent"""
        raise NotImplementedError("Each agent must implement process method")


In [92]:
# ============================================================================
# SPECIALIZED AGENTS
# ============================================================================

class NewsAnalysisAgent(BaseAgent):
    """Analyzes financial news sentiment and impact"""

    def __init__(self):
        super().__init__("News Analysis Agent")
        self.system_prompt = """You are a financial news analyst specializing in sentiment analysis.

INSTRUCTIONS:
1. Analyze news articles objectively
2. Consider both positive and negative aspects
3. Provide a sentiment score from -1 (very negative) to +1 (very positive)
4. Identify key factors driving the sentiment
5. Assess potential stock price impact

EXAMPLE OUTPUT:
{
  "sentiment_score": 0.75,
  "analysis": "Strong positive sentiment driven by earnings beat and product launch",
  "key_factors": ["Earnings exceeded expectations", "New product well-received"],
  "confidence": 0.85
}

Return ONLY valid JSON with keys: sentiment_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process news data for sentiment analysis"""
        start_time = time.time()

        try:
            ticker = data.get('ticker', 'AAPL')
            news_articles = data.get('news', [])

            # Data validation
            if not news_articles:
                logger.warning(f"{self.agent_name}: No news data available")
                return AgentResponse(
                    agent_name=self.agent_name,
                    analysis="No news data available for analysis",
                    score=0.0,
                    confidence=0.0,
                    key_factors=["No news data available"],
                    timestamp=datetime.now().isoformat(),
                    execution_time=time.time() - start_time
                )

            # Prepare news summary
            news_summary = "\n".join([
                f"- {article.get('title', '')}: {article.get('summary', '')}"
                for article in news_articles[:5]
            ])

            user_message = f"""Analyze the following recent news about {ticker}:

{news_summary}

Provide sentiment analysis and impact assessment."""

            # Call LLM
            llm_response, token_usage = self.call_llm(self.system_prompt, user_message)

            # Parse response
            try:
                result = json.loads(llm_response)
                score = result.get('sentiment_score', 0)
                analysis = result.get('analysis', llm_response)
                key_factors = result.get('key_factors', [])

                confidence_raw = result.get('confidence', 0.7)
                try:
                    confidence = max(0.0, min(1.0, float(confidence_raw)))
                except (ValueError, TypeError):
                    confidence = 0.7

            except json.JSONDecodeError:
                logger.warning(f"{self.agent_name}: Failed to parse JSON response")
                score = 0
                analysis = llm_response
                key_factors = ["Unable to parse structured response"]
                confidence = 0.5

            execution_time = time.time() - start_time

            # Add to memory
            self.add_to_memory({
                'input': {'ticker': ticker, 'news_count': len(news_articles)},
                'output': analysis,
                'metadata': {'score': score, 'confidence': confidence}
            })

            # Share insight with other agents
            self.share_insight('news_sentiment', score)
            self.share_insight('news_confidence', confidence)

            # Update metrics
            self.update_metrics(success=True, execution_time=execution_time)

            return AgentResponse(
                agent_name=self.agent_name,
                analysis=analysis,
                score=float(score),
                confidence=float(confidence),
                key_factors=key_factors,
                timestamp=datetime.now().isoformat(),
                execution_time=execution_time,
                token_usage=token_usage
            )

        except Exception as e:
            execution_time = time.time() - start_time
            self.update_metrics(success=False, execution_time=execution_time)
            logger.error(f"{self.agent_name} error: {e}")
            raise

In [93]:
class EarningsAnalysisAgent(BaseAgent):
    """Analyzes earnings reports and financial statements"""

    def __init__(self):
        super().__init__("Earnings Analysis Agent")
        self.system_prompt = """You are a financial analyst specializing in earnings and fundamental analysis.

INSTRUCTIONS:
1. Analyze company financial metrics objectively
2. Compare actuals vs expectations
3. Assess fundamental strength from -1 (very weak) to +1 (very strong)
4. Identify key financial drivers
5. Evaluate growth trends

EXAMPLE OUTPUT:
{
  "fundamental_score": 0.80,
  "analysis": "Strong fundamentals with revenue and EPS beating expectations",
  "key_factors": ["Revenue beat by 5%", "EPS exceeded estimates", "Margin expansion"],
  "confidence": 0.88
}

Return ONLY valid JSON with keys: fundamental_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process earnings and financial data"""
        start_time = time.time()

        try:
            ticker = data.get('ticker', 'UNKNOWN')
            financials = data.get('financials', {})

            # Check for shared insights from other agents
            news_sentiment = self.shared_context.get_insight('news_sentiment')
            if news_sentiment:
                logger.info(f"{self.agent_name}: Considering news sentiment = {news_sentiment}")

            financial_summary = f"""
Company: {ticker}
Revenue: ${financials.get('revenue', 'N/A')}B
EPS: ${financials.get('eps', 'N/A')}
Revenue Growth: {financials.get('revenue_growth', 'N/A')}%
Profit Margin: {financials.get('profit_margin', 'N/A')}%
Expected Revenue: ${financials.get('expected_revenue', 'N/A')}B
Expected EPS: ${financials.get('expected_eps', 'N/A')}
"""

            user_message = f"""Analyze the following financial data for {ticker}:

{financial_summary}

Assess fundamental strength and growth prospects."""

            llm_response, token_usage = self.call_llm(self.system_prompt, user_message)

            try:
                result = json.loads(llm_response)
                score = result.get('fundamental_score', 0)
                analysis = result.get('analysis', llm_response)
                key_factors = result.get('key_factors', [])

                confidence_raw = result.get('confidence', 0.8)
                try:
                    confidence = max(0.0, min(1.0, float(confidence_raw)))
                except (ValueError, TypeError):
                    confidence = 0.8

            except json.JSONDecodeError:
                logger.warning(f"{self.agent_name}: Failed to parse JSON response")
                score = 0
                analysis = llm_response
                key_factors = ["Unable to parse structured response"]
                confidence = 0.6

            execution_time = time.time() - start_time

            self.add_to_memory({
                'input': {'ticker': ticker, 'financials': financials},
                'output': analysis,
                'metadata': {'score': score, 'confidence': confidence}
            })

            self.share_insight('fundamental_score', score)
            self.share_insight('fundamental_confidence', confidence)

            self.update_metrics(success=True, execution_time=execution_time)

            return AgentResponse(
                agent_name=self.agent_name,
                analysis=analysis,
                score=float(score),
                confidence=float(confidence),
                key_factors=key_factors,
                timestamp=datetime.now().isoformat(),
                execution_time=execution_time,
                token_usage=token_usage
            )

        except Exception as e:
            execution_time = time.time() - start_time
            self.update_metrics(success=False, execution_time=execution_time)
            logger.error(f"{self.agent_name} error: {e}")
            raise



In [94]:
class MarketSignalsAgent(BaseAgent):
    """Performs technical analysis on market data"""

    def __init__(self):
        super().__init__("Market Signals Agent")
        self.system_prompt = """You are a technical analyst specializing in market signals and price patterns.

INSTRUCTIONS:
1. Analyze technical indicators objectively
2. Assess technical strength from -1 (very bearish) to +1 (very bullish)
3. Identify support/resistance levels
4. Evaluate trend direction and momentum
5. Consider volume patterns

EXAMPLE OUTPUT:
{
  "technical_score": 0.65,
  "analysis": "Bullish technical setup with price above key moving averages",
  "key_factors": ["Price above 50-day MA", "RSI indicates strength", "Volume confirming uptrend"],
  "confidence": 0.75
}

Return ONLY valid JSON with keys: technical_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process technical market data"""
        start_time = time.time()

        try:
            ticker = data.get('ticker', 'UNKNOWN')
            technicals = data.get('technicals', {})

            # Validate data
            if not technicals.get('current_price') or technicals.get('current_price') == 'N/A':
                logger.warning(f"{self.agent_name}: Insufficient technical data")
                return AgentResponse(
                    agent_name=self.agent_name,
                    analysis="Insufficient technical data for analysis",
                    score=0.0,
                    confidence=0.0,
                    key_factors=["Missing price data"],
                    timestamp=datetime.now().isoformat(),
                    execution_time=time.time() - start_time
                )

            technical_summary = f"""
Ticker: {ticker}
Current Price: ${technicals.get('current_price', 'N/A')}
50-day MA: ${technicals.get('ma_50', 'N/A')}
200-day MA: ${technicals.get('ma_200', 'N/A')}
RSI: {technicals.get('rsi', 'N/A')}
MACD: {technicals.get('macd', 'N/A')}
Volume: {technicals.get('volume', 'N/A')} (Avg: {technicals.get('avg_volume', 'N/A')})
Support: ${technicals.get('support', 'N/A')}
Resistance: ${technicals.get('resistance', 'N/A')}
"""

            user_message = f"""Analyze the following technical data for {ticker}:

{technical_summary}

Assess technical strength and price momentum."""

            llm_response, token_usage = self.call_llm(self.system_prompt, user_message)

            try:
                result = json.loads(llm_response)
                score = result.get('technical_score', 0)
                analysis = result.get('analysis', llm_response)
                key_factors = result.get('key_factors', [])

                confidence_raw = result.get('confidence', 0.7)
                try:
                    confidence = max(0.0, min(1.0, float(confidence_raw)))
                except (ValueError, TypeError):
                    confidence = 0.7

            except json.JSONDecodeError:
                logger.warning(f"{self.agent_name}: Failed to parse JSON response")
                score = 0
                analysis = llm_response
                key_factors = ["Unable to parse structured response"]
                confidence = 0.5

            execution_time = time.time() - start_time

            self.add_to_memory({
                'input': {'ticker': ticker, 'technicals': technicals},
                'output': analysis,
                'metadata': {'score': score, 'confidence': confidence}
            })

            self.share_insight('technical_score', score)
            self.share_insight('technical_confidence', confidence)

            self.update_metrics(success=True, execution_time=execution_time)

            return AgentResponse(
                agent_name=self.agent_name,
                analysis=analysis,
                score=float(score),
                confidence=float(confidence),
                key_factors=key_factors,
                timestamp=datetime.now().isoformat(),
                execution_time=execution_time,
                token_usage=token_usage
            )

        except Exception as e:
            execution_time = time.time() - start_time
            self.update_metrics(success=False, execution_time=execution_time)
            logger.error(f"{self.agent_name} error: {e}")
            raise

In [95]:
class RiskAssessmentAgent(BaseAgent):
    """Assesses investment risk and portfolio fit"""

    def __init__(self):
        super().__init__("Risk Assessment Agent")
        self.system_prompt = """You are a risk management analyst specializing in portfolio risk assessment.

INSTRUCTIONS:
1. Analyze risk metrics objectively
2. Provide risk level score from 0 (very low risk) to 1 (very high risk)
3. Identify key risk factors
4. Assess portfolio diversification implications
5. Evaluate risk-adjusted returns

EXAMPLE OUTPUT:
{
  "risk_score": 0.45,
  "analysis": "Moderate risk profile with acceptable volatility and strong Sharpe ratio",
  "key_factors": ["Beta of 1.15 indicates moderate volatility", "Strong Sharpe ratio", "Manageable drawdown"],
  "confidence": 0.82
}

Return ONLY valid JSON with keys: risk_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process risk metrics"""
        start_time = time.time()

        try:
            ticker = data.get('ticker', 'UNKNOWN')
            risk_data = data.get('risk_metrics', {})

            risk_summary = f"""
Ticker: {ticker}
Beta: {risk_data.get('beta', 'N/A')}
Volatility (30-day): {risk_data.get('volatility', 'N/A')}%
Value at Risk (5%): ${risk_data.get('var_5', 'N/A')}
Sharpe Ratio: {risk_data.get('sharpe_ratio', 'N/A')}
Max Drawdown: {risk_data.get('max_drawdown', 'N/A')}%
Sector Correlation: {risk_data.get('sector_correlation', 'N/A')}
P/E Ratio: {risk_data.get('pe_ratio', 'N/A')}
"""

            user_message = f"""Analyze the following risk metrics for {ticker}:

{risk_summary}

Assess overall investment risk and portfolio implications."""

            llm_response, token_usage = self.call_llm(self.system_prompt, user_message)

            try:
                result = json.loads(llm_response)
                score = result.get('risk_score', 0.5)
                analysis = result.get('analysis', llm_response)
                key_factors = result.get('key_factors', [])

                confidence_raw = result.get('confidence', 0.8)
                try:
                    confidence = max(0.0, min(1.0, float(confidence_raw)))
                except (ValueError, TypeError):
                    confidence = 0.8

            except json.JSONDecodeError:
                logger.warning(f"{self.agent_name}: Failed to parse JSON response")
                score = 0.5
                analysis = llm_response
                key_factors = ["Unable to parse structured response"]
                confidence = 0.6

            execution_time = time.time() - start_time

            self.add_to_memory({
                'input': {'ticker': ticker, 'risk_metrics': risk_data},
                'output': analysis,
                'metadata': {'score': score, 'confidence': confidence}
            })

            self.share_insight('risk_score', score)
            self.share_insight('risk_confidence', confidence)

            self.update_metrics(success=True, execution_time=execution_time)

            return AgentResponse(
                agent_name=self.agent_name,
                analysis=analysis,
                score=float(score),
                confidence=float(confidence),
                key_factors=key_factors,
                timestamp=datetime.now().isoformat(),
                execution_time=execution_time,
                token_usage=token_usage
            )

        except Exception as e:
            execution_time = time.time() - start_time
            self.update_metrics(success=False, execution_time=execution_time)
            logger.error(f"{self.agent_name} error: {e}")
            raise



In [96]:
class SynthesisAgent(BaseAgent):
    """Combines insights from all agents into final recommendation"""

    def __init__(self):
        super().__init__("Research Synthesis Agent")
        self.system_prompt = """You are a senior investment analyst who synthesizes multiple analyses into actionable recommendations.

INSTRUCTIONS:
1. Review all agent analyses objectively
2. Weigh different factors appropriately
3. Provide clear investment recommendation (STRONG BUY, BUY, HOLD, SELL, STRONG SELL)
4. State confidence level (0 to 1)
5. Summarize key reasoning
6. Note important risks

EXAMPLE OUTPUT:
{
  "recommendation": "BUY",
  "confidence": 0.78,
  "analysis": "Strong fundamentals and positive technical signals support a buy recommendation despite moderate risk",
  "key_points": ["Earnings beat expectations", "Technical breakout", "Acceptable risk profile"],
  "risks": ["Market volatility", "Sector headwinds"]
}

Return ONLY valid JSON with keys: recommendation, confidence, analysis, key_points, risks"""

    def process(self, agent_responses: List[AgentResponse]) -> AgentResponse:
        """Synthesize all agent responses"""
        start_time = time.time()

        try:
            # Check shared insights
            all_insights = self.get_shared_insights()
            logger.info(f"{self.agent_name}: Reviewing {len(all_insights)} shared insights")

            analyses_summary = "\n\n".join([
                f"{resp.agent_name}:\n"
                f"Score: {resp.score}\n"
                f"Confidence: {resp.confidence}\n"
                f"Analysis: {resp.analysis}\n"
                f"Key Factors: {', '.join(resp.key_factors)}"
                for resp in agent_responses
            ])

            user_message = f"""Synthesize the following analyses into a final investment recommendation:

{analyses_summary}

Provide comprehensive investment recommendation with supporting reasoning."""

            llm_response, token_usage = self.call_llm(self.system_prompt, user_message)

            try:
                result = json.loads(llm_response)
                recommendation = result.get('recommendation', 'HOLD')
                analysis = result.get('analysis', llm_response)
                key_factors = result.get('key_points', [])

                confidence_raw = result.get('confidence', 0.7)
                try:
                    confidence = max(0.0, min(1.0, float(confidence_raw)))
                except (ValueError, TypeError):
                    confidence = 0.7

                rec_to_score = {
                    'STRONG BUY': 1.0,
                    'BUY': 0.6,
                    'HOLD': 0.0,
                    'SELL': -0.6,
                    'STRONG SELL': -1.0
                }
                score = rec_to_score.get(recommendation, 0.0)

            except json.JSONDecodeError:
                logger.warning(f"{self.agent_name}: Failed to parse JSON response")
                score = 0
                analysis = llm_response
                key_factors = ["Unable to parse structured response"]
                confidence = 0.6

            execution_time = time.time() - start_time

            self.add_to_memory({
                'input': {'num_agents': len(agent_responses)},
                'output': analysis,
                'metadata': {'recommendation': recommendation, 'confidence': confidence}
            })

            self.share_insight('final_recommendation', recommendation)
            self.share_insight('final_score', score)

            self.update_metrics(success=True, execution_time=execution_time)

            return AgentResponse(
                agent_name=self.agent_name,
                analysis=analysis,
                score=float(score),
                confidence=float(confidence),
                key_factors=key_factors,
                timestamp=datetime.now().isoformat(),
                execution_time=execution_time,
                token_usage=token_usage
            )

        except Exception as e:
            execution_time = time.time() - start_time
            self.update_metrics(success=False, execution_time=execution_time)
            logger.error(f"{self.agent_name} error: {e}")
            raise


In [97]:
class CritiqueAgent(BaseAgent):
    """Reviews and validates analysis quality"""

    def __init__(self):
        super().__init__("Critique & Validation Agent")
        self.system_prompt = """You are a critique analyst who reviews investment recommendations for biases, logical errors, and completeness.

INSTRUCTIONS:
1. Review the synthesis objectively
2. Identify logical inconsistencies
3. Detect potential biases
4. Note missing considerations
5. Assess data quality
6. Recommend confidence adjustments

EXAMPLE OUTPUT:
{
  "quality_score": 0.82,
  "issues_found": ["Limited macroeconomic analysis"],
  "suggestions": ["Consider Federal Reserve policy impact", "Add sector comparison"],
  "adjusted_confidence": 0.75
}

Return ONLY valid JSON with keys: quality_score, issues_found, suggestions, adjusted_confidence"""

    def process(self, synthesis_response: AgentResponse) -> AgentResponse:
        """Critique the synthesis"""
        start_time = time.time()

        try:
            user_message = f"""Review this investment analysis for quality and completeness:

Recommendation: {synthesis_response.analysis}
Confidence: {synthesis_response.confidence}
Key Factors: {', '.join(synthesis_response.key_factors)}

Identify any issues, biases, or missing elements."""

            llm_response, token_usage = self.call_llm(self.system_prompt, user_message)

            try:
                result = json.loads(llm_response)
                quality_score = result.get('quality_score', 0.7)
                issues = result.get('issues_found', [])
                suggestions = result.get('suggestions', [])
                adjusted_confidence_raw = result.get('adjusted_confidence', synthesis_response.confidence)

                try:
                    adjusted_confidence = max(0.0, min(1.0, float(adjusted_confidence_raw)))
                except (ValueError, TypeError):
                    adjusted_confidence = synthesis_response.confidence

                analysis = f"Quality Score: {quality_score}\n"
                if issues:
                    analysis += f"Issues Found: {', '.join(issues)}\n"
                if suggestions:
                    analysis += f"Suggestions: {', '.join(suggestions)}"

                key_factors = issues if issues else ["No major issues found"]

            except json.JSONDecodeError:
                logger.warning(f"{self.agent_name}: Failed to parse JSON response")
                quality_score = 0.7
                analysis = llm_response
                adjusted_confidence = synthesis_response.confidence
                key_factors = ["Unable to parse structured response"]

            execution_time = time.time() - start_time

            self.add_to_memory({
                'input': {'synthesis_confidence': synthesis_response.confidence},
                'output': analysis,
                'metadata': {'quality_score': quality_score, 'adjusted_confidence': adjusted_confidence}
            })

            self.share_insight('quality_score', quality_score)
            self.share_insight('adjusted_confidence', adjusted_confidence)

            self.update_metrics(success=True, execution_time=execution_time)

            return AgentResponse(
                agent_name=self.agent_name,
                analysis=analysis,
                score=float(quality_score),
                confidence=float(adjusted_confidence),
                key_factors=key_factors,
                timestamp=datetime.now().isoformat(),
                execution_time=execution_time,
                token_usage=token_usage
            )

        except Exception as e:
            execution_time = time.time() - start_time
            self.update_metrics(success=False, execution_time=execution_time)
            logger.error(f"{self.agent_name} error: {e}")
            raise

In [98]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def print_agent_metrics():
    """Print performance metrics for all agents"""
    print("\n" + "="*80)
    print("AGENT PERFORMANCE METRICS")
    print("="*80)

    agents = [
        NewsAnalysisAgent(),
        EarningsAnalysisAgent(),
        MarketSignalsAgent(),
        RiskAssessmentAgent(),
        SynthesisAgent(),
        CritiqueAgent()
    ]

    for agent in agents:
        metrics = agent.get_metrics()
        print(f"\n{metrics.agent_name}:")
        print(f"  Total Calls: {metrics.total_calls}")
        print(f"  Success Rate: {metrics.successful_calls}/{metrics.total_calls}")
        print(f"  Avg Execution Time: {metrics.average_execution_time:.2f}s")
        print(f"  Total Tokens Used: {metrics.total_tokens_used}")


def print_shared_insights():
    """Print all shared insights between agents"""
    print("\n" + "="*80)
    print("SHARED INSIGHTS")
    print("="*80)

    insights = shared_context.get_all_insights()
    for key, data in insights.items():
        print(f"\n{key}: {data['value']}")
        print(f"  From: {data['from_agent']}")
        print(f"  Time: {data['timestamp']}")

In [99]:
# -*- coding: utf-8 -*-
"""notebooks/01_data_ingestion.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1J_syHGmJHhA8XcAtxw7nQ0KPWMJbnSkd
"""

#import os
os.environ["ALPHAVANTAGE_API_KEY"] = "BVGUKZR1MHVS0T6B"

import os
import requests
import pandas as pd
import yfinance as yf
from datasets import Dataset

# ------------------------------
# Helper: Convert Pandas → Hugging Face Dataset
# ------------------------------
def to_hf(df, schema=None):
    """Convert a pandas DataFrame to a Hugging Face Dataset. Handles empty gracefully."""
    if df is None or getattr(df, "empty", True):
        if schema:
            return Dataset.from_dict({c: [] for c in schema})
        return Dataset.from_dict({})
    if schema:
        df = df[[c for c in schema if c in df.columns]].copy()
    return Dataset.from_pandas(df.reset_index(drop=True), preserve_index=False)

# ------------------------------
# Alpha Vantage Connector (for news + indicators only)
# ------------------------------
class AlphaConnector:
    def __init__(self, api_key=None):
        # Pick up API key from os.environ if not passed directly
        self.api_key = api_key or os.getenv("ALPHAVANTAGE_API_KEY")
        if not self.api_key:
            raise ValueError("Alpha Vantage API key not found. Set os.environ['ALPHAVANTAGE_API_KEY'].")

        self.base_url = "https://www.alphavantage.co/query"

    def fetch_news(self, symbol):
        """Fetch company news & sentiment (Alpha Vantage)."""
        params = {
            "function": "NEWS_SENTIMENT",
            "tickers": symbol,
            "apikey": self.api_key
        }
        r = requests.get(self.base_url, params=params)
        data = r.json()

        if "feed" not in data:
            print("No news data:", data)
            return pd.DataFrame()

        rows = []
        for item in data["feed"]:
            rows.append({
                "published_at": item.get("time_published"),
                "source": item.get("source"),
                "title": item.get("title"),
                "summary": item.get("summary"),
                "url": item.get("url"),
                "overall_sentiment": item.get("overall_sentiment_label"),
                # ** Added By Ali **
                "overall_sentiment_score": item.get("overall_sentiment_score") # both label and score so later agents (NewsAnalysisAgent, SynthesisAgent, etc.) can use either
            })
        return pd.DataFrame(rows)

    def fetch_indicator(self, symbol, indicator, interval="daily", time_period=14, series_type="close"):
        """Generic technical indicator fetch (SMA, RSI, MACD)."""
        params = {
            "function": indicator,
            "symbol": symbol,
            "interval": interval,
            "time_period": time_period,
            "series_type": series_type,
            "apikey": self.api_key
        }
        r = requests.get(self.base_url, params=params)
        data = r.json()

        key_map = {
            "SMA": "Technical Analysis: SMA",
            "RSI": "Technical Analysis: RSI",
            "MACD": "Technical Analysis: MACD"
        }
        key = key_map.get(indicator)
        if key not in data:
            print(f"{indicator} fetch failed:", data)
            return pd.DataFrame()

        df = pd.DataFrame.from_dict(data[key], orient="index")
        df.index = pd.to_datetime(df.index)
        df.reset_index(inplace=True)
        df = df.rename(columns={"index": "date"})

        # Cast numeric values
        for col in df.columns:
            if col != "date":
                df[col] = df[col].astype(float)

        return df

# ------------------------------
# Data Ingestion Manager
# ------------------------------
class DataIngestionManager:
    def __init__(self, api_key=None):
        self.alpha = AlphaConnector(api_key)

    def fetch_all(self, symbol, start=None, end=None):
        """Fetch prices (Yahoo), news (Alpha Vantage), SMA, RSI (Alpha Vantage)."""
        datasets = {}

        # Prices from Yahoo Finance (unlimited)
        try:
            df_prices = yf.download(symbol, start=start, end=end, progress=False)

            # Flatten MultiIndex columns if necessary
            if isinstance(df_prices.columns, pd.MultiIndex):
                df_prices.columns = [c[0].lower() for c in df_prices.columns]

            df_prices = df_prices.reset_index().rename(columns={
                "Date": "date",
                "open": "open",
                "high": "high",
                "low": "low",
                "close": "close",
                "adj close": "adj_close",
                "volume": "volume"
            })
            df_prices["date"] = df_prices["date"].astype(str)

            datasets["prices"] = to_hf(
                df_prices, schema=["date","open","high","low","close","adj_close","volume"]
            )
        except Exception as e:
            print("Yahoo Finance fetch failed:", e)
            datasets["prices"] = to_hf(pd.DataFrame(), schema=["date","open","high","low","close","adj_close","volume"])

        # News from Alpha Vantage
        datasets["news"] = to_hf(
            self.alpha.fetch_news(symbol),
            schema=["published_at","source","title","summary","url","overall_sentiment"]
        )

        # Technical Indicators from Alpha Vantage
        datasets["sma"] = to_hf(
            self.alpha.fetch_indicator(symbol, "SMA", time_period=20),
            schema=["date","SMA"]
        )
        datasets["rsi"] = to_hf(
            self.alpha.fetch_indicator(symbol, "RSI", time_period=14),
            schema=["date","RSI"]
        )

        # Removed MACD to avoid premium-only error
        return datasets

from datetime import datetime, timedelta

mgr = DataIngestionManager()  # will pick up the key from os.environ
symbol = "AAPL"
start = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
end   = datetime.now().strftime("%Y-%m-%d")

datasets = mgr.fetch_all(symbol, start, end)

print("Prices sample:")
print(datasets["prices"].to_pandas().head())

print("News sample:")
print(datasets["news"].to_pandas().head())

/tmp/ipython-input-4032339001.py:120: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_prices = yf.download(symbol, start=start, end=end, progress=False)


No news data: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
SMA fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
RSI fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Prices sample:
         date        open        high         low       close     volume
0  2025-09-18  239.970001  241.199997  236.649994  237.880005   44249600
1  2025-09-19  241.229996  246

In [100]:
# ============================================================================
# INTEGRATION TEST CELL - Engineer 1 + Engineer 3
# ============================================================================

def test_engineer1_engineer3():
    """Test  Engineer 1 and Engineer 3 code"""

    print("="*80)
    print("INTEGRATION TEST - Engineer 1 + Engineer 3")
    print("="*80)

    from datetime import datetime, timedelta

    # ========================================================================
    # STEP 1: Fetch data
    # ========================================================================

    print("\n[STEP 1] Fetching data ...")

    mgr = DataIngestionManager()
    symbol = "AAPL"
    start = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
    end = datetime.now().strftime("%Y-%m-%d")

    datasets = mgr.fetch_all(symbol, start, end)

    # Convert to pandas
    prices_df = datasets["prices"].to_pandas()
    news_df = datasets["news"].to_pandas()
    sma_df = datasets["sma"].to_pandas()
    rsi_df = datasets["rsi"].to_pandas()

    print(f"Fetched data for {symbol}")
    print(f"  Prices: {len(prices_df)} rows")
    print(f"  News: {len(news_df)} articles")
    print(f"  SMA: {len(sma_df)} points")
    print(f"  RSI: {len(rsi_df)} points")

    # ========================================================================
    # STEP 2: Prepare inputs for agents
    # ========================================================================

    print("\n[STEP 2] Preparing data for agents...")

    # News input
    news_articles = news_df.head(5).to_dict('records') if len(news_df) > 0 else []
    news_input = {'ticker': symbol, 'news': news_articles}

    # Technical input
    latest_price = prices_df.iloc[-1] if len(prices_df) > 0 else {}
    latest_sma = sma_df.iloc[-1] if len(sma_df) > 0 else {}
    latest_rsi = rsi_df.iloc[-1] if len(rsi_df) > 0 else {}

    technicals_input = {
        'ticker': symbol,
        'technicals': {
            'current_price': str(latest_price.get('close', 'N/A')),
            'ma_50': str(latest_sma.get('SMA', 'N/A')),
            'rsi': str(latest_rsi.get('RSI', 'N/A')),
            'volume': str(latest_price.get('volume', 'N/A'))
        }
    }

    # Financial input (mock data)
    financials_input = {
        'ticker': symbol,
        'financials': {
            'revenue': '394.3',
            'eps': '6.42',
            'revenue_growth': '15.0',
            'profit_margin': '26.3'
        }
    }

    # Risk input (mock data)
    risk_input = {
        'ticker': symbol,
        'risk_metrics': {
            'beta': '1.15',
            'volatility': '22.5',
            'sharpe_ratio': '1.35'
        }
    }

    print("Data prepared for all agents")

    # ========================================================================
    # STEP 3: Run with agents
    # ========================================================================

    print("\n[STEP 3] Running Engineer 3 agents...")

    # Agent 1: News
    news_agent = NewsAnalysisAgent()
    news_resp = news_agent.process(news_input)
    print(f"  News Analysis: {news_resp.score:+.2f} (confidence: {news_resp.confidence:.0%})")

    # Agent 2: Earnings
    earnings_agent = EarningsAnalysisAgent()
    earn_resp = earnings_agent.process(financials_input)
    print(f"  Earnings Analysis: {earn_resp.score:+.2f} (confidence: {earn_resp.confidence:.0%})")

    # Agent 3: Technical
    market_agent = MarketSignalsAgent()
    market_resp = market_agent.process(technicals_input)
    print(f"  Technical Analysis: {market_resp.score:+.2f} (confidence: {market_resp.confidence:.0%})")

    # Agent 4: Risk
    risk_agent = RiskAssessmentAgent()
    risk_resp = risk_agent.process(risk_input)
    print(f"  Risk Assessment: {risk_resp.score:.2f} (confidence: {risk_resp.confidence:.0%})")

    # ========================================================================
    # STEP 4: Synthesize with synthesis agent
    # ========================================================================

    print("\n[STEP 4] Synthesizing recommendation...")

    all_responses = [news_resp, earn_resp, market_resp, risk_resp]
    synthesis_agent = SynthesisAgent()
    final_resp = synthesis_agent.process(all_responses)
    print(f"  Synthesis Score: {final_resp.score:+.2f} (confidence: {final_resp.confidence:.0%})")

    # ========================================================================
    # STEP 5: Critique with critique agent
    # ========================================================================

    print("\n[STEP 5] Critiquing analysis...")

    critique_agent = CritiqueAgent()
    critique_resp = critique_agent.process(final_resp)
    print(f"  Quality Score: {critique_resp.score:.2f}")
    print(f"  Adjusted Confidence: {critique_resp.confidence:.0%}")

    # ========================================================================
    # FINAL RESULTS
    # ========================================================================

    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80)

    print(f"\nStock: {symbol}")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    print(f"\nAgent Scores:")
    for resp in all_responses:
        print(f"  {resp.agent_name:30s} {resp.score:+.2f}")

    # Determine recommendation
    score = final_resp.score
    if score >= 0.6:
        rec = "STRONG BUY" if score >= 0.8 else "BUY"
    elif score <= -0.6:
        rec = "STRONG SELL" if score <= -0.8 else "SELL"
    else:
        rec = "HOLD"

    print(f"\nFinal Recommendation: {rec}")
    print(f"   Score: {final_resp.score:+.2f}")
    print(f"   Confidence: {final_resp.confidence:.0%}")
    print(f"   Quality: {critique_resp.score:.2f}/1.0")

    print(f"\nAnalysis:")
    print(f"   {final_resp.analysis[:200]}...")

    print(f"\nKey Factors:")
    for i, factor in enumerate(final_resp.key_factors[:3], 1):
        print(f"   {i}. {factor}")

    print("\n" + "="*80)
    print("✓ TEST COMPLETE!")
    print("="*80)

    return {
        'symbol': symbol,
        'recommendation': rec,
        'score': final_resp.score,
        'confidence': final_resp.confidence,
        'quality': critique_resp.score,
        'all_responses': all_responses,
        'synthesis': final_resp,
        'critique': critique_resp
    }


# ============================================================================
# RUN THE TEST
# ============================================================================

print("\nStarting Integration Test...\n")

result = test_engineer1_engineer3()

if result:
    print(f"\nSUCCESS! integration works! ")
    print(f"\nQuick Summary:")
    print(f"  • Recommendation: {result['recommendation']}")
    print(f"  • Final Score: {result['score']:+.2f}")
    print(f"  • Confidence: {result['confidence']:.0%}")
    print(f"  • Quality: {result['quality']:.2f}/1.0")


Starting Integration Test...

INTEGRATION TEST - Engineer 1 + Engineer 3

[STEP 1] Fetching data ...


/tmp/ipython-input-4032339001.py:120: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_prices = yf.download(symbol, start=start, end=end, progress=False)


No news data: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
SMA fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


RSI fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Fetched data for AAPL
  Prices: 22 rows
  News: 0 articles
  SMA: 0 points
  RSI: 0 points

[STEP 2] Preparing data for agents...
Data prepared for all agents

[STEP 3] Running Engineer 3 agents...
  News Analysis: +0.00 (confidence: 0%)
  Earnings Analysis: +0.85 (confidence: 80%)
  Technical Analysis: -1.00 (confidence: 10%)
  Risk Assessment: 0.40 (confidence: 75%)

[STEP 4] Synthesizing recommendation...
  Synthesis Score: +0.00 (confidence: 65%)

[STEP 5] Critiquing analysis...
  Quality Score: 0.60
  Adjusted Confidence: 50%

FINAL RESULTS

Stock: AAPL
Date: 2025-10-18 00:52:37

Agent Scores:
  News Analysis Agent            +0.00
  Earnings Analysis Agent        +0.85
  Market Signals Agent           -1.00
  Risk